In [1]:
# Install dependencies
!pip install pdfplumber PyMuPDF camelot-py[cv] pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.9 MB/s eta 0:00:00


In [2]:
import os
import fitz  # PyMuPDF
import pdfplumber
import camelot
import json
import re

In [15]:
# ==== MAIN EXTRACTION FUNCTION ====
def extract_pdf_content(pdf_path, output_dir, json_name="output.json", charts_dir="charts"):
    """
    Extracts structured content (paragraphs, tables, charts) from a PDF
    and saves as JSON. Chart images are extracted as PNG files.

    pdf_path: path to input PDF
    output_dir: path to save results
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    charts_path = os.path.join(output_dir, charts_dir)
    if not os.path.exists(charts_path):
        os.makedirs(charts_path)

    json_path = os.path.join(output_dir, json_name)

    document_structure = {"pages": []}
    pdf = fitz.open(pdf_path)

    with pdfplumber.open(pdf_path) as plumber_pdf:
        for page_number, page in enumerate(plumber_pdf.pages, start=1):
            page_data = {"page_number": page_number, "content": []}

            # --- Extract Text (Paragraphs) ---
            text = page.extract_text() or ""
            if text.strip():
                sections = split_into_sections(text)
                for sec in sections:
                    page_data["content"].append({
                        "type": "paragraph",
                        "section": sec.get("section"),
                        "sub_section": sec.get("sub_section"),
                        "text": sec.get("content")
                    })

            # --- Extract Tables ---
            try:
                tables = camelot.read_pdf(pdf_path, pages=str(page_number))
                for i, table in enumerate(tables):
                    page_data["content"].append({
                        "type": "table",
                        "section": f"Table {i+1}",
                        "description": None,
                        "table_data": table.df.values.tolist()
                    })
            except Exception:
                pass

            # --- Detect Images (Charts) ---
            page_fitz = pdf[page_number - 1]
            image_list = page_fitz.get_images(full=True)
            if image_list:
                for i, img in enumerate(image_list, start=1):
                    xref = img[0]
                    pix = fitz.Pixmap(pdf, xref)
                    if pix.n > 4:  # Convert CMYK to RGB
                        pix = fitz.Pixmap(fitz.csRGB, pix)
                    chart_filename = f"chart_page{page_number}_{i}.png"
                    chart_path = os.path.join(charts_path, chart_filename)
                    pix.save(chart_path)
                    page_data["content"].append({
                        "type": "chart",
                        "section": f"Chart {i}",
                        "file": chart_filename,
                        "description": "Extracted image (possible chart/figure)"
                    })

            document_structure["pages"].append(page_data)

    # --- Save JSON ---
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(document_structure, f, indent=4, ensure_ascii=False)

    print(f" Extraction complete! JSON saved to {json_path}, charts saved in {charts_path}")


# ==== HELPER FUNCTION ====
def split_into_sections(text):
    """
    Splits text into sections and sub-sections using regex rules.
    """
    sections = []
    lines = text.split("\n")
    current_section = None
    current_sub_section = None
    buffer = []

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Match section headings (1., 2., 3.)
        if re.match(r'^\d+\.\s', line):
            if buffer:
                sections.append({
                    "section": current_section,
                    "sub_section": current_sub_section,
                    "content": " ".join(buffer).strip()
                })
                buffer = []
            current_section = line
            current_sub_section = None

        # Match sub-sections (1.1, 2.3.1, etc.)
        elif re.match(r'^\d+(\.\d+)+\s', line):
            if buffer:
                sections.append({
                    "section": current_section,
                    "sub_section": current_sub_section,
                    "content": " ".join(buffer).strip()
                })
                buffer = []
            current_sub_section = line

        else:
            buffer.append(line)

    if buffer:
        sections.append({
            "section": current_section or "General",
            "sub_section": current_sub_section,
            "content": " ".join(buffer).strip()
        })

    return sections

In [16]:
# Path to my PDF
pdf_file = "/content/[Fund Factsheet - May]360ONE-MF-May 2025.pdf"

# Path of my Google Drive where results will be saved
output_dir = "/content/drive/MyDrive/PDF_Extraction"

# Run the extraction
extract_pdf_content(
    pdf_file,
    output_dir,
    json_name="fund_factsheet.json",
    charts_dir="charts"
)


 Extraction complete! JSON saved to /content/drive/MyDrive/PDF_Extraction/fund_factsheet.json, charts saved in /content/drive/MyDrive/PDF_Extraction/charts
